In [2]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import scipy.stats as stats

In [3]:
data = pd.read_csv('인적사항 전처리2.csv', encoding = 'euc-kr')

In [4]:
data

,Unnamed: 0,사고일시,요일,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,나이,연령대,월,사고시간대
0,0,2007년 1월 11일 01시,4,포장 - 건조,맑음,단일로 - 기타,승용,남,51세,0.0,51세,경상,51.0,50,1,1
1,1,2007년 2월 14일 20시,3,포장 - 건조,맑음,단일로 - 기타,승용,남,40세,0.0,70세,사망,70.0,70,2,20
2,2,2008년 3월 5일 00시,3,포장 - 건조,맑음,단일로 - 기타,승용,남,57세,0.0,57세,중상,57.0,50,3,0
3,3,2008년 3월 19일 15시,3,포장 - 건조,맑음,단일로 - 기타,승용,남,63세,0.0,62세,중상,62.0,60,3,15
4,4,2008년 4월 7일 16시,1,포장 - 건조,맑음,단일로 - 횡단보도부근,이륜,남,40세,0.0,85세,경상,85.0,80,4,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9125,9127,2020년 9월 14일 15시,1,포장 - 건조,맑음,단일로 - 기타,승합,남,74세,1.0,76세,중상,76.0,70,9,15
9126,9128,2020년 9월 23일 22시,3,포장 - 건조,맑음,단일로 - 기타,이륜,남,19세,1.0,59세,중상,59.0,50,9,22
9127,9129,2020년 10월 10일 23시,6,포장 - 건조,맑음,교차로 - 교차로부근,승용,여,37세,0.0,39세,경상,39.0,30,10,23
9128,9130,2020년 11월 24일 01시,2,포장 - 건조,흐림,교차로 - 교차로안,승용,남,54세,0.0,58세,경상,58.0,50,11,1


In [5]:
# 사고일시를 월과 시간으로 나누고, 피해운전자 연령을 연령대로 나누었습니다.
# 그 후, 독립성검정을 위해 필요한 컬럼만 뽑아 데이터셋을 만들었습니다.

In [6]:
col = ['월','사고시간대', '요일', '노면상태', '기상상태', '도로형태', '가해운전자 차종', '피해운전자 성별',
       '연령대','피해운전자 상해정도']

In [7]:
df = pd.DataFrame( columns = col, data = data)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9130 entries, 0 to 9129
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   월           9130 non-null   int64  
 1   사고시간대       9130 non-null   int64  
 2   요일          9130 non-null   int64  
 3   노면상태        9130 non-null   object 
 4   기상상태        9130 non-null   object 
 5   도로형태        9130 non-null   object 
 6   가해운전자 차종    9130 non-null   object 
 7   피해운전자 성별    9130 non-null   float64
 8   연령대         9130 non-null   int64  
 9   피해운전자 상해정도  9130 non-null   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 713.4+ KB


In [9]:
df.head()

,월,사고시간대,요일,노면상태,기상상태,도로형태,가해운전자 차종,피해운전자 성별,연령대,피해운전자 상해정도
0,1,1,4,포장 - 건조,맑음,단일로 - 기타,승용,0.0,50,경상
1,2,20,3,포장 - 건조,맑음,단일로 - 기타,승용,0.0,70,사망
2,3,0,3,포장 - 건조,맑음,단일로 - 기타,승용,0.0,50,중상
3,3,15,3,포장 - 건조,맑음,단일로 - 기타,승용,0.0,60,중상
4,4,16,1,포장 - 건조,맑음,단일로 - 횡단보도부근,이륜,0.0,80,경상


In [10]:
# 변수채택 = 가해운전자 차종, 도로형태, 요일, 피해운전자 성별, 연령대, 사고시간대, 월

In [12]:
## 피해운전자 상해정도와 변수들의 교차표 생성

ctab1 = pd.crosstab(df['가해운전자 차종'],df['피해운전자 상해정도']) 
ctab2 = pd.crosstab(df['도로형태'],df['피해운전자 상해정도']) 
ctab3 = pd.crosstab(df['기상상태'],df['피해운전자 상해정도']) 
ctab4 = pd.crosstab(df['노면상태'],df['피해운전자 상해정도']) 
ctab5 = pd.crosstab(df['요일'],df['피해운전자 상해정도']) 
ctab6 = pd.crosstab(df['피해운전자 성별'],df['피해운전자 상해정도']) 
ctab7 = pd.crosstab(df['연령대'],df['피해운전자 상해정도']) 
ctab8 = pd.crosstab(df['사고시간대'],df['피해운전자 상해정도']) 
ctab9 = pd.crosstab(df['월'],df['피해운전자 상해정도']) 

In [13]:
# 피해운전자 상해정도와 가해운전자 차종, 도로형태, 기상상태, 노면상태, 요일, 피해운전자 성별, 피해운전자 연령대, 사고시간대, 사고 월에 대하여 독립성 검정을 통해 
# 독립성을 확인했습니다

In [14]:
chi2, p, fe, _=stats.chi2_contingency(ctab1)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 447.66805505925794 p: 1.6669619022912897e-72 자유도: 36


In [15]:
chi2, p, fe, _=stats.chi2_contingency(ctab2)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 117.28276973456533 p: 1.3897045503358734e-08 자유도: 44


In [16]:
chi2, p, fe, _=stats.chi2_contingency(ctab3)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 18.724286270194057 p: 0.5398072072696161 자유도: 20


In [17]:
chi2, p, fe, _=stats.chi2_contingency(ctab4)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 36.03253310246504 p: 0.28537646832493807 자유도: 32


In [18]:
chi2, p, fe, _=stats.chi2_contingency(ctab5)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 63.352371558041334 p: 2.1165834394580605e-05 자유도: 24


In [19]:
chi2, p, fe, _=stats.chi2_contingency(ctab6)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 44.36361237737601 p: 5.391520293253985e-09 자유도: 4


In [20]:
chi2, p, fe, _=stats.chi2_contingency(ctab7)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 703.7633639316833 p: 8.686439067533684e-125 자유도: 36


In [21]:
chi2, p, fe, _=stats.chi2_contingency(ctab8)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 339.8992531774982 p: 4.0810717626255205e-30 자유도: 92


In [22]:
chi2, p, fe, _=stats.chi2_contingency(ctab9)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 96.42553937831465 p: 8.650344184056168e-06 자유도: 44


**독립변수를 묶은 후 독립성검정**
* 월 > 계절
* 사고시간대 > 주간, 야간
* 요일 > 평일, 주말
* 가해운전자 차종 > 승용, 승합, 이륜, 기타, 특수차
* 도로형태 > 단일로, 교차로, 기타



In [23]:
df.head()

,월,사고시간대,요일,노면상태,기상상태,도로형태,가해운전자 차종,피해운전자 성별,연령대,피해운전자 상해정도
0,1,1,4,포장 - 건조,맑음,단일로 - 기타,승용,0.0,50,경상
1,2,20,3,포장 - 건조,맑음,단일로 - 기타,승용,0.0,70,사망
2,3,0,3,포장 - 건조,맑음,단일로 - 기타,승용,0.0,50,중상
3,3,15,3,포장 - 건조,맑음,단일로 - 기타,승용,0.0,60,중상
4,4,16,1,포장 - 건조,맑음,단일로 - 횡단보도부근,이륜,0.0,80,경상


In [24]:

car_1 = ['승용']
car_2 = ['이륜','자전거','개인형이동수단(PM)']
car_3 = ['승합']
car_4 = ['화물', '원동기', '건설기계']
car_5 = ['기타불명', '특수']

사고시간대를 주간/야간으로 변경

In [25]:
list_a = []

In [26]:
for i in range(len(df['사고시간대'])):
  if df['사고시간대'][i] >= 18:
    list_a.append('야간')
  elif df['사고시간대'][i] >= 6:
    list_a.append('주간')
  else:
    list_a.append('야간')
print(list_a) 

['야간', '야간', '야간', '주간', '주간', '야간', '주간', '야간', '야간', '야간', '야간', '야간', '야간', '주간', '야간', '주간', '야간', '야간', '주간', '주간', '야간', '야간', '주간', '주간', '야간', '주간', '주간', '야간', '주간', '야간', '주간', '주간', '주간', '야간', '주간', '야간', '주간', '주간', '야간', '주간', '주간', '야간', '야간', '주간', '야간', '주간', '주간', '주간', '야간', '야간', '주간', '주간', '주간', '야간', '야간', '야간', '야간', '주간', '야간', '주간', '야간', '야간', '야간', '야간', '주간', '주간', '야간', '야간', '야간', '주간', '주간', '주간', '주간', '주간', '야간', '야간', '야간', '주간', '야간', '주간', '야간', '주간', '야간', '야간', '주간', '주간', '주간', '야간', '주간', '야간', '주간', '주간', '주간', '야간', '야간', '주간', '주간', '주간', '주간', '주간', '주간', '야간', '야간', '야간', '야간', '주간', '야간', '야간', '야간', '야간', '주간', '야간', '야간', '주간', '야간', '주간', '야간', '주간', '야간', '야간', '야간', '주간', '주간', '야간', '야간', '주간', '야간', '주간', '주간', '야간', '야간', '야간', '주간', '야간', '주간', '주간', '야간', '주간', '야간', '야간', '주간', '주간', '야간', '야간', '주간', '주간', '주간', '주간', '주간', '주간', '주간', '야간', '주간', '주간', '주간', '야간', '주간', '야간', '주간', '주간', '야간', '주간', '야간', '주간', '주간', '야간', '주간

In [27]:
df['주간야간'] = list_a

요일을 주말(1) / 평일(0)로 변경

In [28]:
list_b = []

In [29]:
for i in range(len(df['요일'])):
  if df['요일'][i] >= 6:
    list_b.append('1') # 주말 = 1
  else:
    list_b.append('0') # 평일 = 0
print(list_b) 

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0',

In [30]:
df['주말'] = list_b

월을 계절로 변경

In [31]:
list_c = []

In [32]:
for i in range(len(df['월'])):
  if df['월'][i] in [12,1,2]:
    list_c.append('겨울') 
  elif df['월'][i] in [3,4,5]:
    list_c.append('봄')
  elif df['월'][i] in [6,7,8]:
    list_c.append('여름')
  else:
    list_c.append('가을')
print(list_c) 

['겨울', '겨울', '봄', '봄', '봄', '봄', '여름', '가을', '가을', '겨울', '가을', '겨울', '겨울', '봄', '여름', '가을', '가을', '겨울', '봄', '봄', '가을', '여름', '여름', '여름', '가을', '봄', '봄', '봄', '여름', '겨울', '봄', '여름', '여름', '겨울', '봄', '여름', '가을', '겨울', '겨울', '겨울', '봄', '가을', '겨울', '겨울', '겨울', '여름', '가을', '겨울', '겨울', '봄', '봄', '가을', '가을', '봄', '봄', '가을', '여름', '여름', '가을', '겨울', '봄', '봄', '가을', '겨울', '겨울', '봄', '여름', '가을', '겨울', '여름', '겨울', '봄', '가을', '가을', '가을', '겨울', '겨울', '겨울', '겨울', '여름', '여름', '가을', '겨울', '겨울', '겨울', '봄', '봄', '봄', '가을', '가을', '여름', '여름', '가을', '가을', '가을', '겨울', '봄', '봄', '봄', '여름', '가을', '겨울', '봄', '여름', '여름', '가을', '가을', '가을', '여름', '여름', '여름', '가을', '가을', '가을', '겨울', '겨울', '겨울', '봄', '여름', '여름', '여름', '여름', '여름', '가을', '가을', '가을', '가을', '봄', '여름', '가을', '겨울', '봄', '봄', '봄', '가을', '봄', '여름', '겨울', '봄', '가을', '가을', '겨울', '봄', '여름', '가을', '겨울', '여름', '여름', '가을', '가을', '겨울', '봄', '여름', '여름', '가을', '가을', '겨울', '여름', '여름', '여름', '가을', '가을', '봄', '여름', '여름', '여름', '겨울', '가을', '가을', '봄', '봄', '여름', '여름', '

In [33]:
df['계절'] = list_c

도로형태를 단일로 / 교차로 / 기타로 구분

In [34]:
list_d = []

In [35]:
for i in range(len(df['월'])):
  if '교차로' in df['도로형태'][i]:
    list_d.append('교차로')
  elif '단일로' in df['도로형태'][i]:
    list_d.append('단일로')
  else:
    list_d.append('기타')
print(list_d)

['단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '교차로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '단일로', '단일로', '단일로', '교차로', '교차로', '교차로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '단일로', '교차로', '단일로', '단일로', '단일로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '교차로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '교차로', '교차로', '단일로', '교차로', '단일로', '단일로', '단일로', '단일로', '단일로', '단일로'

In [36]:
df['도로형태구분'] = list_d

가해자차종 구분

In [37]:
list_e = []
car_1 = ['승용']
car_2 = ['이륜','자전거','개인형이동수단(PM)']
car_3 = ['승합']
car_4 = ['화물', '원동기', '건설기계']
car_5 = ['기타불명', '특수']

In [38]:
for i in range(len(df['월'])):
  if df['가해운전자 차종'][i] in car_1:
    list_e.append('승용') 
  elif df['가해운전자 차종'][i] in car_2:
    list_e.append('이륜')
  elif df['가해운전자 차종'][i] in car_3:
    list_e.append('승합')
  elif df['가해운전자 차종'][i] in car_4:
    list_e.append('화물')
  else:
    list_e.append('기타')
print(list_e) 

['승용', '승용', '승용', '승용', '이륜', '승용', '승용', '승합', '이륜', '승용', '이륜', '승용', '승용', '승용', '승합', '화물', '화물', '승용', '승합', '승용', '승합', '승용', '승용', '승용', '승용', '승용', '화물', '승용', '이륜', '이륜', '승용', '승용', '이륜', '승합', '화물', '승용', '화물', '화물', '승합', '승용', '승용', '승용', '화물', '승용', '승용', '승용', '승용', '승용', '승용', '승용', '기타', '승용', '승용', '승용', '승용', '승용', '승용', '이륜', '화물', '승용', '승용', '승합', '승용', '승용', '승용', '승용', '승용', '이륜', '승용', '화물', '승용', '승용', '화물', '승용', '승용', '승용', '승용', '승용', '화물', '화물', '승용', '승용', '승용', '승용', '이륜', '승용', '승합', '승용', '승용', '승용', '승용', '승용', '화물', '승용', '승용', '승합', '승용', '승용', '승용', '승용', '승용', '승합', '기타', '승용', '승용', '이륜', '승용', '승용', '승용', '승용', '이륜', '화물', '승용', '승용', '화물', '이륜', '승용', '화물', '승용', '승용', '이륜', '화물', '화물', '승용', '승용', '화물', '화물', '승용', '승용', '승용', '이륜', '이륜', '승용', '승용', '화물', '승용', '승용', '승합', '승용', '승용', '승용', '화물', '승용', '승용', '승합', '이륜', '승용', '승용', '화물', '승용', '이륜', '승용', '승용', '승합', '승합', '이륜', '승용', '승용', '승용', '승용', '승합', '승용', '승용', '승용', '승용', '화물', '승용

In [39]:
df['차종구분'] = list_e

In [40]:
df_set = df[['차종구분','계절','도로형태구분','주간야간','주말','연령대','피해운전자 성별','피해운전자 상해정도']]

# **묶은 변수들로 독립성검정**

In [41]:
ctab_1 = pd.crosstab(df['차종구분'],df['피해운전자 상해정도']) 
ctab_2 = pd.crosstab(df['도로형태구분'],df['피해운전자 상해정도']) 
ctab_3 = pd.crosstab(df['주말'],df['피해운전자 상해정도']) 
ctab_4 = pd.crosstab(df['계절'],df['피해운전자 상해정도']) 
ctab_5 = pd.crosstab(df['주간야간'],df['피해운전자 상해정도'])  

In [42]:
chi2, p, fe, _=stats.chi2_contingency(ctab_1)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 260.52329887555567 p: 3.574059810217073e-46 자유도: 16


In [43]:
chi2, p, fe, _=stats.chi2_contingency(ctab_2)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 35.401173616187414 p: 2.2593766856894695e-05 자유도: 8


In [44]:
chi2, p, fe, _=stats.chi2_contingency(ctab_3)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 10.18532794201704 p: 0.03741893006023743 자유도: 4


In [45]:
chi2, p, fe, _=stats.chi2_contingency(ctab_4)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 54.243033186289935 p: 2.472647122240122e-07 자유도: 12


In [46]:
chi2, p, fe, _=stats.chi2_contingency(ctab_5)
print('통계량:',chi2,'p:', p,'자유도:', fe)

통계량: 36.61737354498332 p: 2.1596884279140782e-07 자유도: 4


# **인코딩**

In [81]:
df_set3['피해운전자 상해정도'] = df_set3['피해운전자 상해정도'].map({'상해없음':0,'부상신고':0,'경상':1,'중상':1,'사망':2})

In [68]:
df_set3['주간야간'] = df_set3['주간야간'].map({'주간':0,'야간':1})

In [63]:
df_set1 = pd.get_dummies(data = df_set, columns = ['도로형태구분'], prefix = '도로형태구분')

In [64]:
df_set2 = pd.get_dummies(data = df_set1, columns = ['계절'], prefix = '계절')

In [65]:
df_set3 = pd.get_dummies(data = df_set2, columns = ['차종구분'], prefix = '차종구분')

In [82]:
df_set3.head()

,주간야간,주말,연령대,피해운전자 성별,피해운전자 상해정도,도로형태구분_교차로,도로형태구분_기타,도로형태구분_단일로,계절_가을,계절_겨울,계절_봄,계절_여름,차종구분_기타,차종구분_승용,차종구분_승합,차종구분_이륜,차종구분_화물
0,1,0,50,0.0,1,0,0,1,0,1,0,0,0,1,0,0,0
1,1,0,70,0.0,2,0,0,1,0,1,0,0,0,1,0,0,0
2,1,0,50,0.0,1,0,0,1,0,0,1,0,0,1,0,0,0
3,0,0,60,0.0,1,0,0,1,0,0,1,0,0,1,0,0,0
4,0,0,80,0.0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [103]:
df_set3['주말'] = df_set3['주말'].astype(int)

# **오버샘플링**

In [104]:
X=df_set3.drop("피해운전자 상해정도",axis=1)
y=df_set3["피해운전자 상해정도"]

In [105]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,train_size=0.8,random_state=1)

In [106]:
from imblearn.over_sampling import SMOTE
#SMOTE 오버샘플링이 한 기법으로 낮은 비율로 존재하는 클래스의 데이터를 최근접 이웃 알고리즘을
#활용하여 새롭게 생성하는 기법

In [107]:
from imblearn.over_sampling import SMOTE
#SMOTE 오버샘플링이 한 기법으로 낮은 비율로 존재하는 클래스의 데이터를 최근접 이웃 알고리즘을
#활용하여 새롭게 생성하는 기법

smote = SMOTE(random_state=42)
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 값의 분포 :\n',pd.Series(y_train_over).value_counts() )

SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (7304, 16) (7304,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : (19692, 16) (19692,)
SMOTE 적용 후 값의 분포 :
 1    6564
2    6564
0    6564
Name: 피해운전자 상해정도, dtype: int64
